In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn.preprocessing import MinMaxScaler


In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:
df = pd.read_csv("energy_data.csv")


In [ ]:
energy_scaled=scaler.fit_transform(df[['Energy Consumption (kWh)']])

In [ ]:


def create_sequences(data, time_steps=24):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)


In [ ]:
TIME_STEPS = 24

X, y = create_sequences(energy_scaled, TIME_STEPS)


In [ ]:
split_index = int(0.8 * len(X))

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]


In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test  = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=(TIME_STEPS, 1)))
model.add(Dropout(0.2))

model.add(LSTM(32))
model.add(Dropout(0.2))

model.add(Dense(1))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error'
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 55s 24ms/step - loss: 0.0579 - val_loss: 0.0584
Epoch 2/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step - loss: 0.0581 - val_loss: 0.0584
Epoch 3/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 52s 23ms/step - loss: 0.0571 - val_loss: 0.0584
Epoch 4/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 84s 24ms/step - loss: 0.0579 - val_loss: 0.0584
Epoch 5/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 80s 23ms/step - loss: 0.0581 - val_loss: 0.0583
Epoch 6/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 52s 23ms/step - loss: 0.0577 - val_loss: 0.0583
Epoch 7/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 86s 25ms/step - loss: 0.0579 - val_loss: 0.0584
Epoch 8/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 55s 24ms/step - loss: 0.0580 - val_loss: 0.0583
Epoch 9/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 54s 24ms/step - loss: 0.0580 - val_loss: 0.0583
Epoch 10/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 81s 24ms/step - loss: 0.0581 - val_loss: 0.0583
Epoch 11/30
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 53s 23ms/step - loss: 0.0582 - val_loss: 0.0583
Epoch 12

In [ ]:
y_pred_scaled = model.predict(X_test)


625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_actual = scaler.inverse_transform(y_pred_scaled)


In [ ]:
mae = mean_absolute_error(y_test_actual, y_pred_actual)
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))

print("LSTM MAE:", mae)
print("LSTM RMSE:", rmse)


LSTM MAE: 0.884235073244078
LSTM RMSE: 1.186279535128576


In [ ]:
model.save("lstm_energy_model.h5")
